## 🏆 RFM기반으로 고객 세분화 분석을 진행 
### 1) 재구매율 T / Y 고객으로 분류
### 2) 고객 5등급별 UPGRADE 혹은 STAY 고객으로 분류함 
#### 2 - 1) 상위 25%를 기준

In [13]:
import pandas as pd
import numpy as np
customer = pd.read_csv('C:/Users/zxoro/OneDrive/바탕 화면/더스포츠커뮤니케이션 자료/jupyter/customer_info_new.csv')

In [4]:
customer

,membership_no,game_date,game_time,match_info,product_grade_name,rank,sale,new_price,reserve_diff,MB_AGE_x,favorite_idx_x,customer_type,month,day,weather,survey_y,re_y
0,102806,2022-02-19,16:30:00,수원 삼성블루윙즈,W1~W5 구역(World Class석),2,Y,19000,5,26,박창환,Platinum,2,토,구름많음,T,T
1,138215,2022-02-19,16:30:00,수원 삼성블루윙즈,E석 1층(Exciting석),2,N,13000,5,15,오반석,Platinum,2,토,구름많음,T,T
2,138215,2022-02-19,16:30:00,수원 삼성블루윙즈,E석 1층(Exciting석),2,N,19500,5,15,오반석,Platinum,2,토,구름많음,T,T
3,125933,2022-02-19,16:30:00,수원 삼성블루윙즈,S석/N석,2,N,12000,2,20,이명주,Platinum,2,토,구름많음,T,T
4,143862,2022-02-19,16:30:00,수원 삼성블루윙즈,W1~W5 구역(World Class석),2,N,57000,2,26,무고사,Bad,2,토,구름많음,T,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24801,136617,2022-10-16,19:00:00,포항 스틸러스,W1~W5 구역(World Class석),4,N,10500,5,41,이명주,Gold,10,일,흐림,T,T
24802,136617,2022-10-16,19:00:00,포항 스틸러스,W1~W5 구역(World Class석),4,N,63000,5,41,이명주,Gold,10,일,흐림,T,T
24803,136617,2022-10-16,19:00:00,포항 스틸러스,W1~W5 구역(World Class석),4,N,42000,4,41,이명주,Gold,10,일,흐림,T,T
24804,149857,2022-10-16,19:00:00,포항 스틸러스,W1~W5 구역(World Class석),4,Y,42000,3,30,에르난데스,Gold,10,일,흐림,T,T


In [5]:
customer = customer.drop(['game_time', 'new_price', 'reserve_diff','favorite_idx_x', 'month', 'day', 'weather'], axis = 1)

📍 혜택

In [8]:
sale = pd.read_csv('C:/Users/zxoro/OneDrive/바탕 화면/더스포츠커뮤니케이션 자료/jupyter/sale.csv')
sale

a = pd.merge(left = sale, right = customer, how = 'right', on = 'membership_no')
df = a.drop_duplicates().reset_index(drop =  True)

📍 나이 구간

In [9]:
# 나이 구간 나누기 
def age_categorize(age):
    age = (age // 10) * 10
    return age

df['MB_AGE_x'] = df['MB_AGE_x'].apply(age_categorize)

In [10]:
df.shape

(52564, 11)

In [9]:
# cluster = pd.read_csv('C:/Users/zxoro/OneDrive/바탕 화면/더스포츠커뮤니케이션 자료/jupyter/cluster.csv')
# cluster = cluster[['membership_no', 'clusters']]
# cluster

# df = pd.merge(df, cluster, how = 'left',on = ['membership_no'])

# ✨고객 세분화 진행

In [34]:
rfm = pd.read_csv('C:/Users/zxoro/OneDrive/바탕 화면/더스포츠커뮤니케이션 자료/jupyter/rfm_analysis.csv')

In [35]:
rfm

,membership_no,recency,frequency,monetary,recency_score,frequency_score,monetary_score,score,customer_type
0,137,141,1,100000,3,1,3,7,Bronze
1,153,1,11,164000,5,5,4,14,Platinum
2,180,1,2,61000,5,2,3,10,Silver
3,208,1,22,281000,5,5,5,15,Platinum
4,291,1,29,609000,5,5,5,15,Platinum
...,...,...,...,...,...,...,...,...,...
7408,151972,1,1,40000,5,1,2,8,Bronze
7409,151978,1,1,14000,5,1,1,7,Bronze
7410,151980,1,1,14000,5,1,1,7,Bronze
7411,151981,1,1,30000,5,1,2,8,Bronze


In [16]:
rfm['state'] = np.nan * 7413

In [17]:
def function(a, b, c , type_ = 'Bad', df = rfm):
    test = df[df['customer_type'] == type_].drop(['membership_no', 'monetary', 'customer_type', 'recency', 'score', 'frequency'], axis = 1)
    test['recency_score'] = a * test['recency_score']
    test['frequency_score'] = b * test['frequency_score']
    test['monetary_score'] = c * test['monetary_score']
   # 서로 다른 행별 평균
    test['compare'] = test.apply(np.mean, 1).round(2)
    return test

In [18]:
# Bad고객은 사용빈도가 중요하다고 판단되어 가중치를 다르게 부여하였음 
Bad = function(0.3, 0.4, 0.3,type_ = 'Bad', df = rfm)
# 최빈값 = 0.75% 기준으로 했음 0.63
print(Bad['compare'].describe())
print(Bad['compare'].mode())

# 당신들은 승급요건에 충족되셨습니다 ~!!!!!!!!!!
Bad[Bad['compare'] > 0.63]

count    2596.000000
mean        0.523636
std         0.111460
min         0.330000
25%         0.430000
50%         0.530000
75%         0.630000
max         0.700000
Name: compare, dtype: float64
0    0.63
Name: compare, dtype: float64


,recency_score,frequency_score,monetary_score,state,compare
103,0.9,0.8,0.3,NaN,0.67
151,0.6,0.8,0.6,NaN,0.67
164,0.6,0.8,0.6,NaN,0.67
185,0.3,1.2,0.6,NaN,0.70
194,0.9,0.8,0.3,NaN,0.67
...,...,...,...,...,...
5906,0.9,0.8,0.3,NaN,0.67
6004,0.9,0.8,0.3,NaN,0.67
6132,0.9,0.8,0.3,NaN,0.67
6157,0.9,0.8,0.3,NaN,0.67


In [19]:
bad_idx = Bad[Bad['compare'] > 0.63].index
rfm.loc[bad_idx, 'state'] = 'upgrade'

In [20]:
# Bronze고객은 사용빈도가 중요하다고 판단되어 가중치를 다르게 부여하였음 
Bronze = function(0.3, 0.4, 0.3,type_ = 'Bronze', df = rfm)
# 
print(Bronze['compare'].mode())
print(Bronze['compare'].describe())
# mode = 0.73이고 75%는 0.87임 (75명만 충족됨), 중간을 기준으로 0.8로 설정함
# 당신들은 승급요건에 충족되셨습니다 ~!!!!!!!!!!
Bronze[Bronze['compare'] > 0.82]

0    0.73
Name: compare, dtype: float64
count    1788.000000
mean        0.798177
std         0.059302
min         0.730000
25%         0.730000
50%         0.770000
75%         0.870000
max         0.900000
Name: compare, dtype: float64


,recency_score,frequency_score,monetary_score,state,compare
12,0.9,0.8,0.9,NaN,0.87
40,1.2,0.8,0.6,NaN,0.87
43,1.2,0.4,0.9,NaN,0.83
44,1.5,0.8,0.3,NaN,0.87
51,1.2,0.8,0.6,NaN,0.87
...,...,...,...,...,...
7400,1.5,0.4,0.6,NaN,0.83
7402,1.5,0.4,0.6,NaN,0.83
7405,1.5,0.4,0.6,NaN,0.83
7408,1.5,0.4,0.6,NaN,0.83


In [21]:
bronze_idx = Bronze[Bronze['compare'] > 0.82].index
rfm.loc[bronze_idx, 'state'] = 'upgrade'

In [22]:
# Silver고객은 사용빈도가 중요하다고 판단되어 가중치를 다르게 부여하였음 
Silver = function(0.33, 0.33, 0.33,type_ = 'Silver', df = rfm)
# 
print(Silver['compare'].mode())
print(Silver['compare'].describe())
# mode = 0.99이고 75%는 1.21임 충족되는 고객이 없기 때문에 50%를 기준으로 잡음
# 당신들은 승급요건에 충족되셨습니다 ~!!!!!!!!!!
Silver[Silver['compare'] > 1.1]

0    0.99
Name: compare, dtype: float64
count    1584.000000
mean        1.083681
std         0.089934
min         0.990000
25%         0.990000
50%         1.100000
75%         1.210000
max         1.210000
Name: compare, dtype: float64


,recency_score,frequency_score,monetary_score,state,compare
5,1.32,1.32,0.99,NaN,1.21
6,1.32,1.32,0.99,NaN,1.21
15,1.32,1.32,0.99,NaN,1.21
30,1.65,0.99,0.99,NaN,1.21
33,1.65,0.99,0.99,NaN,1.21
...,...,...,...,...,...
7188,1.65,0.99,0.99,NaN,1.21
7225,1.65,0.99,0.99,NaN,1.21
7261,1.65,0.99,0.99,NaN,1.21
7268,1.65,0.66,1.32,NaN,1.21


In [23]:
silver_idx = Silver[Silver['compare'] > 1.1].index
rfm.loc[silver_idx, 'state'] = 'upgrade'

In [24]:
# Gold고객은 좌석 중요하다고 판단되어 가중치를 다르게 부여하였음 
Gold = function(0.3, 0.3, 0.4,type_ = 'Gold', df = rfm)
# 
print(Gold['compare'].mode())
print(Gold['compare'].describe())
# mode = 1.33이고 75%는 1.43임 충족되는 고객이 없기 때문에 75%를 기준으로 잡음
# 당신들은 승급요건에 충족되셨습니다 ~!!!!!!!!!!
Gold[Gold['compare'] > 1.43]

0    1.33
Name: compare, dtype: float64
count    726.000000
mean       1.378884
std        0.058491
min        1.300000
25%        1.330000
50%        1.370000
75%        1.430000
max        1.470000
Name: compare, dtype: float64


,recency_score,frequency_score,monetary_score,state,compare
21,1.2,1.2,2.0,NaN,1.47
29,1.2,1.2,2.0,NaN,1.47
41,1.5,0.9,2.0,NaN,1.47
252,1.2,1.2,2.0,NaN,1.47
288,0.9,1.5,2.0,NaN,1.47
...,...,...,...,...,...
6381,1.5,0.9,2.0,NaN,1.47
6610,1.5,0.9,2.0,NaN,1.47
6975,1.5,0.9,2.0,NaN,1.47
7034,1.5,0.9,2.0,NaN,1.47


In [25]:
gold_idx = Gold[Gold['compare'] > 1.43].index
rfm.loc[gold_idx, 'state'] = 'upgrade'

In [26]:
# Platinum고객은 좌석 중요하다고 판단되어 가중치를 다르게 부여하였음 
# Platinum고객은 점수가 높기 때문에 승급보다는 혜택을 적게 주고 유지시키는 방향으로 분석하기 
Platinum = function(0.3, 0.3, 0.4,type_ = 'Platinum', df = rfm)
# 
print(Platinum['compare'].mode())
print(Platinum['compare'].describe())
# mode = 1.33이고 75%는 1.67임 충족되는 고객이 없기 때문에 75%를 기준으로 잡음
# 당신들은 승급요건에 충족되셨습니다 ~!!!!!!!!!!
Platinum[Platinum['compare'] >= 1.67]

0    1.67
Name: compare, dtype: float64
count    719.000000
mean       1.622768
std        0.058847
min        1.530000
25%        1.570000
50%        1.670000
75%        1.670000
max        1.670000
Name: compare, dtype: float64


,recency_score,frequency_score,monetary_score,state,compare
3,1.5,1.5,2.0,NaN,1.67
4,1.5,1.5,2.0,NaN,1.67
7,1.5,1.5,2.0,NaN,1.67
9,1.5,1.5,2.0,NaN,1.67
10,1.5,1.5,2.0,NaN,1.67
...,...,...,...,...,...
5756,1.5,1.5,2.0,NaN,1.67
6058,1.5,1.5,2.0,NaN,1.67
6059,1.5,1.5,2.0,NaN,1.67
6319,1.5,1.5,2.0,NaN,1.67


In [27]:
platinum_idx = Platinum[Platinum['compare'] >= 1.67].index
rfm.loc[platinum_idx, 'state'] = 'upgrade'

In [28]:
# 마지막 정리하기 
rfm['state'] = rfm['state'].fillna('stay')

In [29]:
rfm

,membership_no,recency,frequency,monetary,recency_score,frequency_score,monetary_score,score,customer_type,state
0,137,141,1,100000,3,1,3,7,Bronze,stay
1,153,1,11,164000,5,5,4,14,Platinum,stay
2,180,1,2,61000,5,2,3,10,Silver,stay
3,208,1,22,281000,5,5,5,15,Platinum,upgrade
4,291,1,29,609000,5,5,5,15,Platinum,upgrade
...,...,...,...,...,...,...,...,...,...,...
7408,151972,1,1,40000,5,1,2,8,Bronze,upgrade
7409,151978,1,1,14000,5,1,1,7,Bronze,stay
7410,151980,1,1,14000,5,1,1,7,Bronze,stay
7411,151981,1,1,30000,5,1,2,8,Bronze,upgrade


In [30]:
state = rfm[['membership_no', 'state']]
# state.to_csv('./state.csv', index = False)

In [10]:
state = pd.read_csv('C:/Users/zxoro/OneDrive/바탕 화면/더스포츠커뮤니케이션 자료/jupyter/state.csv')

In [11]:
state.head()

,membership_no,state
0,137,stay
1,153,stay
2,180,stay
3,208,upgrade
4,291,upgrade


📍upgrade고객과 customer_info가 들어간 데이터 합치기

In [12]:
df = pd.merge(left = df, right = state, how = 'left' , on = 'membership_no')
# 저장
# df.to_csv('./rere.csv', index = False)

In [17]:
df_T = df[df['re_y'] == 'T']
df_T['customer_type'].value_counts(normalize=True)

Platinum    0.660428
Gold        0.152751
Silver      0.123420
Bronze      0.043976
Bad         0.019426
Name: customer_type, dtype: float64

#### 📍 재구매를 하지 않을 고객 중에 upgrade 될 가능성이 있는 고객

In [18]:
df_N = df[df['re_y'] == 'N']
df_N['customer_type'].value_counts(normalize=True)

Bad       0.682728
Bronze    0.290363
Silver    0.026909
Name: customer_type, dtype: float64

In [24]:
# 예매권북 판매 비율을 높이기 ! 
# 10장에 10만원 ==> 10번을 오지 않기 때문에 
# 유지율이 3달이니 예매권북을 차라리 5번으로 ~~?

df_N[(df_N['customer_type'] == 'Silver') & (df_N['survey_y'] == 'T')].sort_values('membership_no')

,product_denomination_name,membership_no,game_date,match_info,product_grade_name,rank,sale,MB_AGE_x,customer_type,survey_y,re_y,state
48877,예매권북,84220,2022-10-16,포항 스틸러스,W6~W10 구역(With석),4,Y,30,Silver,T,N,stay
41489,예매권북,108617,2022-09-14,울산 현대축구단,프리미엄석 클래식,4,Y,40,Silver,T,N,stay
36104,포스코 건설 예매권,149656,2022-08-27,FC서울,W6~W10 구역(With석),3,Y,30,Silver,T,N,stay
48793,"3,000포인트 할인 쿠폰(일반)",150288,2022-10-16,포항 스틸러스,피크닉 테이블석(2인),4,N,20,Silver,T,N,stay
48794,일반,150288,2022-10-16,포항 스틸러스,피크닉 테이블석(2인),4,N,20,Silver,T,N,stay
42717,일반,150959,2022-10-01,울산 현대축구단,하이네켄 테이블석(2인),4,N,30,Silver,T,N,stay
